In [3]:
import json
import os

with open('../API_KEY', 'r') as f:
    os.environ['OPENAI_API_KEY'] = f.read()

In [18]:
from langchain.schema import Document


json_docs = []

with open('train.jsonl', 'r') as f:
    for line in f:
        json_docs.append(json.loads(line))

docs = []
for doc in json_docs:
    text = doc.pop('text')
    doc['source'] = doc['source'].replace('rtdocs', 'https:/').replace('\\', '/')
    docs.append(Document(page_content=text, metadata=doc))
print(len(docs))
print(docs[0])

5175
page_content='.md\n.pdf\ndocument.write(`\n  <button class="theme-switch-button btn btn-sm btn-outline-primary navbar-btn rounded-circle" title="light/dark" aria-label="light/dark" data-bs-placement="bottom" data-bs-toggle="tooltip">\n    <span class="theme-switch" data-mode="light"><i class="fa-solid fa-sun"></i></span>\n    <span class="theme-switch" data-mode="dark"><i class="fa-solid fa-moon"></i></span>\n    <span class="theme-switch" data-mode="auto"><i class="fa-solid fa-circle-half-stroke"></i></span>\n  </button>\n`);\ndocument.write(`\n  <button class="btn btn-sm navbar-btn search-button search-button__button" title="Search" aria-label="Search" data-bs-placement="bottom" data-bs-toggle="tooltip">\n    <i class="fa-solid fa-magnifying-glass"></i>\n  </button>\n`);\nDependents\nDependents#\nDependents stats for hwchase17/langchain\n[update: 2023-06-05; only dependent repositories with Stars > 100]\nRepository\nStars\nopenai/openai-cookbook\n38024\nLAION-AI/Open-Assistant\n

In [33]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
embedder = OpenAIEmbeddings(model='text-embedding-ada-002')


In [34]:
raise  # Only re-run this when necessary 
docstore = FAISS.from_documents(documents=docs, embedding=embedder)
# docstore = FAISS.from_documents(documents=docs[0:2], embedding=embedder)
# from tqdm.auto import tqdm
# for doc in tqdm(docs[2:]):
#     docstore.add_documents([doc])
docstore.save_local('faiss_dbs', 'langchain_rtd')
docstore

In [39]:
docstore = FAISS.load_local('faiss_dbs', embedder, 'langchain_rtd')

In [40]:
from langchain.chains.question_answering import load_qa_chain
# from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

In [41]:
# llm = OpenAI()
llm = ChatOpenAI(temperature=0.0)
chain = load_qa_chain(llm, chain_type='stuff')  # Stuff all data in at once (Other methods might work better if more context required)

In [47]:
chain.llm_chain.prompt

ChatPromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], output_parser=None, partial_variables={}, template="Use the following pieces of context to answer the users question. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n----------------\n{context}", template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], output_parser=None, partial_variables={}, template='{question}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [48]:
query = "What does the ConversationRetrievalChain do?"
qdocs = docstore.similarity_search(query, k=4)
qdocs

[Document(page_content='chain = ConversationalRetrievalChain(\n    retriever=vectorstore.as_retriever(),\n    question_generator=question_generator,\n    combine_docs_chain=doc_chain,\n)\nchat_history = []\nquery = "What did the president say about Ketanji Brown Jackson"\nresult = chain({"question": query, "chat_history": chat_history})\nresult[\'answer\']\n" The president said that Ketanji Brown Jackson is one of the nation\'s top legal minds, a former top litigator in private practice, a former federal public defender, from a family of public school educators and police officers, a consensus builder, and has received a broad range of support from the Fraternal Order of Police to former judges appointed by Democrats and Republicans. \\nSOURCES: ../../state_of_the_union.txt"\nConversationalRetrievalChain with streaming to stdout#\nOutput from the chain will be streamed to stdout token by token in this example.\nfrom langchain.chains.llm import LLMChain\nfrom langchain.callbacks.streami

In [50]:
response = chain.run(input_documents=qdocs, question=query)
response

'The ConversationalRetrievalChain is a tool that can be used to generate context-aware answers to questions. It searches a VectorStore to identify the most relevant code snippets for a given query and uses an LLM (GPT-4) to generate comprehensive, context-aware answers based on retrieved code snippets and conversation history. It can also be customized with different types of combine document chains and question answering chains.'